### Load libraries

In [87]:
from dotenv import load_dotenv
import os

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate

import boto3
import pandas as pd

### Load environment variables
I created .env file where I stored different api keys (AWS and openai).

In [88]:
load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_REGION")

# Set the OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### LLM
I will use chatgpt 3.5 turbo as llm for this tutorial, You can use another llm models.\
[List of llm models that you can use with langchain](https://python.langchain.com/docs/integrations/chat/#all-chat-models)

In [89]:
# Create the OpenAI chat model
chat = ChatOpenAI(
    model="gpt-4o-mini",  # You can use also "gpt-3.5-turbo" / gpt-4 if you want
    temperature=0.2
)

### Data
I will use an S3 cluster data for this tutorial, I will upload data and then push it to postgres database.

##### Read S3

In [90]:
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"]=AWS_SECRET_ACCESS_KEY
# os.environ["AWS_REGION"]= AWS_REGION

# Initialize S3 client
s3_client = boto3.client("s3")

# Function to list files in an S3 bucket
def list_s3_objects(bucket_name):
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    if "Contents" in response:
        return [obj["Key"] for obj in response["Contents"]]
    return "No files found in the bucket."

# Function to read a file from S3
def read_s3_object(bucket_name, file_key):
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    # content = response["Body"].read().decode("utf-8")
    content = pd.read_csv(response["Body"])
    return content


In [91]:
list_s3_objects("earthquakedb")

['data_etl.csv']

In [92]:
df = read_s3_object("earthquakedb", "data_etl.csv")
df.head(10)

,Unnamed: 0,Year,Local_Time,UTC_Time,Magnitude,Location,Depth,Latitude,Longitude,Event_type,Assessment,Agency
0,0,2025,Jan 26 14:48:47,Jan 26 13:48:47,1.3,Martigny VS,4.9,46.16,7.06,NaN,automatic,SED
1,1,2025,Jan 26 09:36:00,Jan 26 08:36:00,1.2,Liestal BL,10.6,47.48,7.74,NaN,automatic,SED
2,2,2025,Jan 26 05:53:30,Jan 26 04:53:30,1.3,Bosco/Gurin TI,5.0,46.30,8.34,NaN,automatic,SED
3,3,2025,Jan 26 05:40:34,Jan 26 04:40:34,1.7,CHATEL-ST-DENIS FR,2.2,46.49,6.89,NaN,automatic,SED
4,4,2025,Jan 25 23:06:29,Jan 25 22:06:29,1.6,Unterschaechen UR,8.5,46.90,8.80,NaN,automatic,SED
5,5,2025,Jan 25 20:50:13,Jan 25 19:50:13,1.2,Aosta I,5.0,45.80,7.43,NaN,automatic,SED
6,6,2025,Jan 25 20:47:20,Jan 25 19:47:20,1.1,Laufenburg,7.9,47.70,7.96,NaN,automatic,SED
7,7,2025,Jan 25 15:23:28,Jan 25 14:23:28,0.3,Eglisau ZH,1.7,47.57,8.54,earthquake,manual,SED
8,8,2025,Jan 25 15:23:22,Jan 25 14:23:22,0.2,Eglisau ZH,1.7,47.57,8.54,earthquake,manual,SED
9,9,2025,Jan 24 16:09:32,Jan 24 15:09:32,1.3,Bourg-Saint-Pierre VS,6.9,45.91,7.04,earthquake,manual,SED


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8747 entries, 0 to 8746
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8747 non-null   int64  
 1   Year        8747 non-null   int64  
 2   Local_Time  8747 non-null   object 
 3   UTC_Time    8747 non-null   object 
 4   Magnitude   8747 non-null   float64
 5   Location    8747 non-null   object 
 6   Depth       8747 non-null   float64
 7   Latitude    8747 non-null   float64
 8   Longitude   8747 non-null   float64
 9   Event_type  8740 non-null   object 
 10  Assessment  8747 non-null   object 
 11  Agency      8747 non-null   object 
dtypes: float64(4), int64(2), object(6)
memory usage: 820.2+ KB


#### Assign Canton name

In [94]:
import geopandas as gpd
from shapely.geometry import Point
from langchain.tools import Tool, StructuredTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel

# Load the Swiss cantons Shapefile
# Ensure you provide the correct path to your Shapefile
shapefile_path = "shp/swissBOUNDARIES3D_1_5_TLM_KANTONSGEBIET.shp"

regions_gdf = gpd.read_file(shapefile_path)
if regions_gdf.crs is None:
        regions_gdf.set_crs("EPSG:4326", inplace=True)  # Assuming WGS84

# Convert points list to a GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(df["Longitude"], df["Latitude"])]
points_gdf = gpd.GeoDataFrame(
        df,geometry=geometry,  # Create Point geometry
        crs="EPSG:4326"  # Set coordinate reference system
)
if regions_gdf.crs != points_gdf.crs:
        regions_gdf = regions_gdf.to_crs(points_gdf.crs)
joined = gpd.sjoin(points_gdf, regions_gdf, how="left", predicate="within")
joined = joined[joined["OBJEKTART"]=="Kanton"]
data = joined[["Unnamed: 0", "Year", "Local_Time", "Magnitude", "Location", "Latitude", "Longitude", "NAME"]]
data.head(10)

,Unnamed: 0,Year,Local_Time,Magnitude,Location,Latitude,Longitude,NAME
0,0,2025,Jan 26 14:48:47,1.3,Martigny VS,46.16,7.06,Valais
1,1,2025,Jan 26 09:36:00,1.2,Liestal BL,47.48,7.74,Basel-Landschaft
3,3,2025,Jan 26 05:40:34,1.7,CHATEL-ST-DENIS FR,46.49,6.89,Vaud
4,4,2025,Jan 25 23:06:29,1.6,Unterschaechen UR,46.90,8.80,Uri
7,7,2025,Jan 25 15:23:28,0.3,Eglisau ZH,47.57,8.54,Zürich
8,8,2025,Jan 25 15:23:22,0.2,Eglisau ZH,47.57,8.54,Zürich
10,10,2025,Jan 24 10:22:39,0.8,Porrentruy JU,47.37,7.06,Jura
16,16,2025,Jan 21 07:21:40,0.3,Faido TI,46.55,8.79,Ticino
19,19,2025,Jan 21 04:08:59,0.2,Porrentruy JU,47.37,6.92,Jura
20,20,2025,Jan 21 01:34:01,0.7,Oberwald VS,46.60,8.36,Bern


##### Load S3 data into postgres table

#### Create database

In [95]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# PostgreSQL credentials
db_host = "localhost"
db_user = "postgres"
db_pass = "postgres"
db_port = "5432"  # Default PostgreSQL port
db_name = "llm_sql"

In [96]:
# Create database
try:
    conn = psycopg2.connect(
        dbname="postgres", user=db_user, password=db_pass, host=db_host, port=db_port
    )
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute(f"CREATE DATABASE {db_name};")
    cursor.close()
    conn.close()
    print(f"Database '{db_name}' created successfully.")
except psycopg2.errors.DuplicateDatabase:
    print(f"Database '{db_name}' already exists.")
except Exception as e:
    print(f"Error creating database: {e}")


Database 'llm_sql' already exists.


##### Create table and push data

In [97]:
from sqlalchemy import create_engine, Column, Integer, String, Float, MetaData, Table, DateTime
from sqlalchemy.exc import SQLAlchemyError

# Create a PostgreSQL engine
engine = create_engine(f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

In [99]:
from sqlalchemy.exc import NoSuchTableError

metadata_obj = MetaData()
table_name = "earthquakes"

try:
    table = Table(table_name, metadata_obj, autoload_with=engine)
    table.drop(engine)
    print(f"Table '{table_name}' deleted successfully.")
except NoSuchTableError:
    print(f"Table '{table_name}' does not exist.")

Table 'earthquakes' does not exist.


In [100]:

# Sample DataFrame
# data = read_s3_object("earthquakedb", "data_etl.csv")
data = data.rename(columns={"Unnamed: 0":   "Index",
                            "Location":     "location",
                            "Magnitude":    "magnitude",
                            "Latitude":     "latitude",
                            "Longitude":    "longitude",
                            "NAME":         "canton"})
data = data[["Index", "Year", "Local_Time", "magnitude", "location", "latitude", "longitude", "canton"]]
data = data[(data["magnitude"]>=2) & (data["Year"]>=2020)]

data["local_Time"] = pd.to_datetime(data["Year"].astype(str) + " " + data["Local_Time"],
                                    format="%Y %b %d %H:%M:%S")
data.drop(columns=["Year", "Local_Time"], inplace=True)

# Define your table schema
earthquake_table = Table(
    "earthquakes", metadata_obj,
    Column("Index", Integer, primary_key=True, autoincrement=True),
    Column("local_time", DateTime),
    Column("magnitude", Float),
    Column("location", String(255)),
    Column("latitude", Float),
    Column("longitude", Float),
    Column("canton", String(255))
)

# Create the table if it does not exist
try:
    metadata_obj.create_all(engine)
    print("Table created (if not exists).")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")

# Push the DataFrame into PostgreSQL (replace table if it exists)
data.to_sql('earthquakes', engine, if_exists='replace', index=False)

print("Data successfully pushed to PostgreSQL!")

Table created (if not exists).
Data successfully pushed to PostgreSQL!


##### Test
To test, I am retrieving first 10 records.

In [101]:
query = "SELECT * FROM earthquakes LIMIT 10;"
pd.read_sql(query, engine)

,Index,magnitude,location,latitude,longitude,canton,local_Time
0,53,2.3,Lausanne VD,46.51,6.74,Vaud,2025-01-15 05:11:13
1,125,2.1,Samnaun GR,46.90,10.31,Graubünden,2025-01-06 15:23:19
2,127,2.3,Cevio TI,46.34,8.55,Ticino,2025-01-06 09:56:49
3,131,2.2,Kreuzlingen TG,47.63,9.21,Thurgau,2025-01-05 16:54:55
4,136,2.0,Savognin GR,46.59,9.57,Graubünden,2025-01-03 15:29:42
5,152,2.2,Boncourt JU,47.48,7.01,Jura,2024-12-31 07:33:28
6,232,2.4,Weinfelden TG,47.60,9.08,Thurgau,2024-12-20 16:17:11
7,278,2.3,Sanetschpass VS,46.33,7.39,Valais,2024-12-13 08:24:30
8,285,2.4,Saas Fee VS,46.05,7.93,Valais,2024-12-12 05:31:24
9,286,2.0,Saas Fee VS,46.05,7.92,Valais,2024-12-12 05:31:10


### SQL Agent

In [102]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.agents import create_sql_agent, AgentExecutor
from langchain.agents.agent_types import AgentType

db = SQLDatabase.from_uri(f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

# Create a SQLDatabase Toolkit (this contains the necessary tools)
toolkit = SQLDatabaseToolkit(db=db, llm=chat)


In [111]:
tools = toolkit.get_tools()
print([tool.name for tool in tools])

['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']


In [104]:
# Define prompt
# prompt = ChatPromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_template(
    "{input}\n\nPrevious Actions:\n{agent_scratchpad}"
)

# Create LangChain SQL agent
sql_agent = create_openai_tools_agent(
    llm=chat,
    tools=tools,
    prompt=prompt,
)

# Create agent executor
agent_executor = AgentExecutor(agent=sql_agent,
                               tools=tools,
                               verbose=False)

### Simple questions

- 10 first records

In [76]:
response1 = agent_executor.invoke({"input": "Display the first 10 records from the earthquakes table."})
print(response1['output'])

Here are the first 10 records from the earthquakes table:

1. ID: 53, Magnitude: 2.3, Location: Lausanne VD, Latitude: 46.51, Longitude: 6.74, Canton: Vaud, Date: 2025-01-15 05:11:13
2. ID: 125, Magnitude: 2.1, Location: Samnaun GR, Latitude: 46.9, Longitude: 10.31, Canton: Graubünden, Date: 2025-01-06 15:23:19
3. ID: 127, Magnitude: 2.3, Location: Cevio TI, Latitude: 46.34, Longitude: 8.55, Canton: Ticino, Date: 2025-01-06 09:56:49
4. ID: 131, Magnitude: 2.2, Location: Kreuzlingen TG, Latitude: 47.63, Longitude: 9.21, Canton: Thurgau, Date: 2025-01-05 16:54:55
5. ID: 136, Magnitude: 2.0, Location: Savognin GR, Latitude: 46.59, Longitude: 9.57, Canton: Graubünden, Date: 2025-01-03 15:29:42
6. ID: 152, Magnitude: 2.2, Location: Boncourt JU, Latitude: 47.48, Longitude: 7.01, Canton: Jura, Date: 2024-12-31 07:33:28
7. ID: 232, Magnitude: 2.4, Location: Weinfelden TG, Latitude: 47.6, Longitude: 9.08, Canton: Thurgau, Date: 2024-12-20 16:17:11
8. ID: 278, Magnitude: 2.3, Location: Sanetschp

- strongest earthquake recorded

In [77]:
response2 = agent_executor.invoke({"input": "Retrieve the record of the earthquake of the most magnitude"})
print(response2['output'])

The earthquake with the highest magnitude recorded in the database occurred at 'Pragelpass GL/SZ' with a magnitude of 4.4. The coordinates for this earthquake are 47.05 latitude and 8.86 longitude. It occurred in the 'Schwyz' canton on June 4, 2024, at 02:34:32.


- Oldest record

In [78]:
response4 = agent_executor.invoke({"input": "when was the oldest record in earthquake table."})
print(response4['output'])

The oldest record in the "earthquakes" table is from January 14, 2020, at 23:47:03.


- Region with most earthquakes

In [105]:
response5 = agent_executor.invoke({"input": "In which swiss Location occurs the most earthquakes."})
print(response5['output'])

The location in Switzerland that experiences the most earthquakes is Elm GL, with a total of 42 recorded earthquakes.


In [106]:
response6 = agent_executor.invoke({"input": "In which swiss canton occurs the most earthquakes"})
print(response6['output'])

The Swiss canton that experiences the most earthquakes is Valais, with a total of 71 recorded earthquakes.


Add new tool

In [107]:
import matplotlib
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

In [119]:
tools = toolkit.get_tools()
tools.append(repl_tool)
print([tool.name for tool in tools])

['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker', 'python_interpreter']


In [122]:
import matplotlib.pyplot as plt
import geopandas as gpd
from langchain_experimental.utilities import PythonREPL
from langchain.tools import Tool

def plot_geodata():
    """
    Executes Python code to generate a geoplot.
    The function assumes a predefined GeoDataFrame `gdf` is available.
    """
    try:
        # Load a sample dataset if gdf is not provided
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

        # Create the plot
        fig, ax = plt.subplots(figsize=(10, 6))
        world.plot(ax=ax, color='lightgrey', edgecolor='black')

        # Save the plot
        plot_path = "geoplot.png"
        plt.savefig(plot_path)
        plt.close()

        return f"GeoPlot saved as {plot_path}"

    except Exception as e:
        return f"Error in plotting: {str(e)}"

# Define the tool
repl_tool = Tool(
    name="geo_plotter",
    description="Generates a geoplot. Assumes geospatial data is available and plots it using GeoPandas & Matplotlib.",
    func=plot_geodata
)

# Add the tool to the SQL agent tools
tools = toolkit.get_tools()
tools.append(repl_tool)

# Verify tools
print([tool.name for tool in tools])


['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker', 'geo_plotter']


In [123]:
# Define prompt
# prompt = ChatPromptTemplate.from_template("{input}")
prompt = ChatPromptTemplate.from_template(
    "{input}\n\nPrevious Actions:\n{agent_scratchpad}"
)

# Create LangChain SQL agent
sql_agent = create_openai_tools_agent(
    llm=chat,
    tools=tools,
    prompt=prompt,
)

# Create agent executor
agent_executor = AgentExecutor(agent=sql_agent,
                               tools=tools,
                               verbose=False)

In [124]:
response7 = agent_executor.invoke({"input": "Count number of earthquakes for each canton and plot it in geoplot"})
print(response7)

TypeError: plot_geodata() takes 0 positional arguments but 1 was given